In [ ]:
from scrapy.selector import Selector
from urllib.parse import urljoin
import requests
import pprint as pp
import json

In [ ]:
# Crawling through all the pages
# Start from page 1, keep going to the next page to scrape untill the last page.

def get_quotes(url):
    selector = Selector(text=requests.get(url).text)
    quotes = selector.css('.quote')
    quote_data = []

    for quote in quotes:
        
        text = quote.css('.text::text').extract_first().replace('“', '').replace('”', '')
        author = quote.css('.author::text').extract_first()
        author_link = url + quote.css('span a::attr(href)').extract_first()
        tags = quote.css('.tags .tag::text').extract()
        
        quote_data.append({
            "text": text,
            "author": author,
            "author_link": author_link,
            "tags": tags
        })
        
    next_page = selector.css('.pager .next a::attr(href)').extract_first()
    
    if(next_page is None):
        next_url = None
    else:
        next_url = urljoin("http://quotes.toscrape.com", next_page)
    
    return (quote_data, next_url)

def get_all_quotes():
    all_quotes_data = []
    current_url = "http://quotes.toscrape.com"
    
    while True:
        data, next_url = get_quotes(current_url)
        all_quotes_data += data
        if(next_url is None): break
            
        current_url = next_url   
        print("Proceed to " + next_url)
        
    return all_quotes_data

# Print all quotes
all_quotes = get_all_quotes()
pp.pprint(all_quotes)

In [ ]:
# Save as local json file "data.json"
with open('quotes_to_scrape.json', 'w') as file:
  json.dump(all_quotes, file, sort_keys=True, indent=4)